In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
data_df = pd.read_csv('energy_efficiency_data.csv')

data_x_df = data_df.drop('Heating Load', axis=1)
data_x_df = pd.get_dummies(data_x_df,
                           columns=['Orientation', 'Glazing Area Distribution'],
                           dtype=float)

data_df = pd.concat(
    [data_x_df, data_df['Heating Load']],
    axis=1,
)

data_df


In [ ]:
dataset = np.array(data_df)
np.random.shuffle(dataset)

train_ratio = 0.75
train_size = int(len(dataset) * train_ratio)

train_feature = dataset[:train_size, :-1]
test_feature = dataset[train_size:, :-1]
train_target = dataset[:train_size, -1:]
test_target = dataset[train_size:, -1:]

In [ ]:
def split_batch(x, y, batch_size):
    for i in range(0, len(x), batch_size):
        if i + batch_size < len(x):
            yield x[i:i + batch_size], y[i:i + batch_size]
        yield x[i:], y[i:]

In [ ]:
from src import layer, losses
from src.model import Model

INPUT_FEATURES = train_feature.shape[1]
OUTPUT_FEATURES = 1

model = Model([
    layer.Linear(INPUT_FEATURES, 16),
    layer.ReLU(),
    layer.Linear(16, 8),
    layer.ReLU(),
    layer.Linear(8, OUTPUT_FEATURES),
])

In [ ]:
mse = losses.MSE()
loss = mse(model(train_feature[:10]), train_target[:10])
loss

In [ ]:
EPOCHES = 10
BATCH_SIZE = 32

for epoch in range(EPOCHES):

    losses = []
    for x, y in split_batch(train_feature, train_target, BATCH_SIZE):
        y_pred = model(x)
        loss = mse(y_pred, y)
        losses.append(loss)
        grad = mse.backward()
        model.backward(grad)
        model.update(0.001)

    print(f'Epoch: {epoch}, Loss: {np.mean(losses)}')